In [38]:
# This is a standalone file to process a Statistics Canada census file for machine learning
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Since we learned what processing needs to be done in the previous file,
# we can now write a function to do this processing

def processCensusData(censusData, year = 2016):
    '''
    This function processes a Statistics Canada census data file for machine learning.
    It handles the different data structures for the 2016 and 2021 census files.
    The function returns a pandas dataframe with the processed data.
    Args:
        censusData: A pandas dataframe containing the census data as imported from the csv file.
        year: The year of the census data. Default is 2016.
    '''
    # If this is 2016 data, let's rename the 'DIM: Profile...' column
    if year == 2016:
        censusData.rename(columns={'DIM: Profile of Census Subdivisions (2247)': 'CHARACTERISTIC_NAME'}, inplace=True)
    # Next we trim the characteristic names
    censusData['CHARACTERISTIC_NAME'] = censusData['CHARACTERISTIC_NAME'].str.replace(r'[^\x00-\x7F]+', '', regex=True)
    censusData['CHARACTERISTIC_NAME'] = censusData['CHARACTERISTIC_NAME'].str.strip()

    # For the 2021 data, we need to split the commnity names on the comma
    # to drop additional subdivision info and match the names with any 2016 data
    if year == 2021:
        censusData['GEO_NAME'] = censusData['GEO_NAME'].str.split(',').str[0]

    # Trim the columns by keywords
    keyWords = ['population', 
                'household', 
                'employment', 
                'industry', 
                'labour', # Labour force information
                'years', # To catch age categories
                'decile', # To catch income decile
                'Employ', # Employment status
                'Unemploy', # Employment status
                'Occupation', # Industry of occupation
                'occupation' # Industry of occupation
                '$', # To catch income values. This will generate a lot of duplicates, but we will handle that later
                ]
    trimmedData = censusData[censusData['CHARACTERISTIC_NAME'].str.contains('|'.join(keyWords), case=False, na=False)]
    # Pivot the table on the 'GEO_NAME' column
    valueColumn = ''
    if year == 2016:
        valueColumn = 'Dim: Sex (3): Member ID: [1]: Total - Sex'
    else: # 2021 data
        valueColumn = 'C1_COUNT_TOTAL'
    pivotedData = trimmedData.pivot_table(index='GEO_NAME', 
                                         columns= 'CHARACTERISTIC_NAME', 
                                         values= valueColumn, 
                                         aggfunc='first')
    # Coerce numeric values
    pivotedData = pivotedData.apply(pd.to_numeric, errors='coerce')
    return pivotedData

# Import the census data from /statCanData
year = 2021
location = 'ON'
fileName = f'{location}_{year}.csv'
encoding = 'ISO-8859-1'
rawData = pd.read_csv(f'statCanData/{fileName}', encoding=encoding)
processedData = processCensusData(rawData, year)

# Save the processed data to a csv file, with a name based on the original file
processedData.to_csv(f'processedData/processed_{fileName}', index=True)
processedData.head()

    

CHARACTERISTIC_NAME,"% of households living in a dwelling provided by the local government, First Nation or Indian band spending more than 30% on shelter costs",% of owner households spending 30% or more of its income on shelter costs,% of owner households with a mortgage,% of tenant households in subsidized housing,% of tenant households spending 30% or more of its income on shelter costs,0 Legislative and senior management occupations,0 to 14 years,0 to 17 years,0 to 17 years (%),0 to 4 years,...,Total - Visible minority for the population in private households - 25% sample data,Total number of census families in private households - 100% data,Total visible minority population,Two-maintainer household,Two-or-more-person non-census-family households,Under 5 years,Unemployed,Unemployment rate,With employment income,Without employment income
GEO_NAME,,,,,,,,,,,,,,,,,,,,,
Abitibi 70,NaN,NaN,NaN,NaN,NaN,0.0,25.0,NaN,NaN,10.0,...,150.0,45.0,10.0,20.0,0.0,0.0,15.0,18.8,NaN,NaN
Addington Highlands,NaN,16.0,47.2,24.4,31.1,0.0,295.0,355.0,14.0,90.0,...,2475.0,765.0,0.0,530.0,45.0,0.0,140.0,15.3,1170.0,1015.0
Adelaide-Metcalfe,NaN,10.9,57.2,0.0,23.5,25.0,600.0,695.0,7.2,175.0,...,2890.0,870.0,40.0,570.0,25.0,55.0,100.0,5.9,1775.0,510.0
Adjala-Tosorontio,NaN,17.0,61.4,0.0,26.4,115.0,1695.0,2095.0,6.0,490.0,...,10955.0,3345.0,740.0,2200.0,90.0,200.0,560.0,9.0,6840.0,2420.0
Admaston/Bromley,NaN,5.0,49.7,0.0,27.3,0.0,550.0,670.0,11.2,150.0,...,2995.0,885.0,35.0,655.0,40.0,30.0,95.0,5.9,1750.0,690.0
